In [1]:
import pandas as pd 

df = pd.read_csv('/home/workspace/DACON/Click_predict/data/train.csv')

In [2]:
print(len(df[df['Click'] == 1]))
print(len(df[df['Click'] == 0]))

5569860
23035531


In [4]:
5569860*0.2417942959508943

1346760.377245048

In [4]:
from sklearn.preprocessing import LabelEncoder

zero_df = df.fillna(0)
X_train = zero_df.drop(columns=['Click', "ID"]) 
y_trian = zero_df['Click'] 

In [8]:
from sklearn.preprocessing import LabelEncoder

# label encoding
encoding_target = list(X_train.dtypes[X_train.dtypes == "object"].index)
for i in encoding_target:
    le = LabelEncoder()
    le.fit(X_train[i])
    X_train[i] = le.transform(X_train[i])


: 

In [3]:
import lightgbm as lgb
import xgboost as xgb
import catboost as cat

lgbm = lgb.LGBMClassifier(device='gpu')

In [6]:
lgbm.fit(X_train, y_trian)

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: ID, F01, F02, F03, F05, F07, F08, F09, F10, F12, F13, F15, F16, F17, F20, F21, F22, F23, F25, F26, F28, F30, F31, F34, F35, F37, F39

In [5]:
import duckdb
import pandas as pd

train_path = '/home/workspace/DACON/Click_predict/data/train.csv'

con = duckdb.connect()

# 전체 데이터에서 0 -> 70% 1 -> 30% 
df = con.query(f"""(SELECT *
                        FROM read_csv_auto('{train_path}')
                        WHERE Click = 0
                        ORDER BY random()
                        LIMIT 10000000)
                        UNION ALL
                        (SELECT *
                        FROM read_csv_auto('{train_path}')
                        WHERE Click = 1
                        ORDER BY random()
                        LIMIT 4000000)""").df()
con.close()


df.head()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,ID,Click,F01,F02,F03,F04,F05,F06,F07,F08,...,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39
0,TRAIN_22317699,0,LLKAVMO,EKJSVRG,None,NaN,YKHABYT,1,ZXRGKTC,FTPHMPQ,...,NZGEZLW,GTISJWW,2.0,1.0,ABEHJLN,IRUDRFB,2.0,LKUECII,0.0,IHRMCYS
1,TRAIN_07635998,0,SLTVYAG,FMNAZRC,OOWQSYU,35.0,ZVSDNXN,0,GDPSRQN,VAWXMCR,...,QKNIEFG,GTISJWW,8.0,8.0,VQBTVRP,IRUDRFB,3.0,IXYBBPA,0.0,PNSPZPK
2,TRAIN_06455937,0,YWULVDO,ZPDZABB,DTZFPRW,1.0,GZNRZYF,184,PQZBVMG,LPYPUNA,...,TFZIBRI,GTISJWW,6948.0,NaN,HQRXMHN,IRUDRFB,NaN,TFJMLCZ,NaN,LLVJPTF
3,TRAIN_19923222,0,LFZKJYF,TIRLRYP,None,NaN,KLXPUID,18,ONXZCWF,FTPHMPQ,...,XYRNHWW,GTISJWW,22993.0,0.0,GSOCZYE,IRUDRFB,NaN,RKZTFJO,0.0,TYUWOJI
4,TRAIN_19315139,0,XOZOJFY,RTRPFPH,IAGJDOH,NaN,GOUWEUZ,-1,GTQRZBT,LPYPUNA,...,TFZIBRI,GTISJWW,5296.0,NaN,YOASCVO,IRUDRFB,NaN,YOQYXUH,0.0,TEUDIMK


In [8]:
14000000*0.3

4200000.0

In [6]:
10000000*0.2417942959508943

2417942.959508943

In [ ]:
7백만


In [8]:
len(df[df['Click'] == 1])

5569860

In [9]:
len(df[df['Click'] == 0])

7000000